In [104]:
using JuMP, Gurobi, CPLEX
using LinearAlgebra

# Problem setting

- single vehicle
- One-to-One PDP
- Nodes: 5 Pickup locations (1 to 5), and 5 Delivery locations (6 to 10), origin(11), destination(12)
- Time Windows: Randomly generated for each pickup and delivery location.
- Stack Limit: 3 (maximum number of items the vehicle can carry).
- Cost Matrix: Randomly generated costs between each pair of locations.

In [132]:
# Number of pickup and delivery requests
num_requests = 5
num_nodes = 2 * num_requests + 2 # Includes two depot nodes

# Define the set of pickup nodes P and delivery nodes D
P = [i for i in 1:num_requests]
D = [i for i in num_requests+1:2*num_requests]
depot_start = 2*num_requests+1 #node 11
depot_end = num_nodes #node 12
vehicle_capacity = 4

# Define time windows for each node (here, we're setting random examples)
time_windows = vcat([(rand(0:30), rand(30:60)) for _ in 1:num_requests], [(rand(0:30), rand(30:80)) for _ in 1:num_requests])
time_windows = vcat(time_windows, [(0, 0), (0, 100)]) # Time window for the initial depot and the final depot


# Define load sizes for pickup and delivery requests (here, we're setting random examples)
loads = [1 for _ in 1:num_requests] # 1 load for pickups
loads = vcat(loads, -loads) # Loads for deliveries are the negated pickups
loads = vcat(loads, [0, 0]) # Loads for depots are zero

# Random service durations at each location
service_durations = rand(1:4, 2*num_requests) # Service durations between 1 to 4 units
service_durations = vcat(service_durations, [0, 0])

# Define a random symmetric travel time matrix respecting triangular inequality
# 노드의 무작위 위치 생성
node_positions = [rand(0:10, 2) for _ in 1:num_nodes-1]
node_positions = vcat(node_positions, [node_positions[11]]) # 마지막 노드는 11 번째 노드와 동일한 위치에 배치

# 유클리드 거리를 사용하여 이동 시간 계산
function calculate_travel_time(pos1, pos2)
    return round(norm(pos1 - pos2)) # 유클리드 거리를 기반으로 이동 시간 계산
end

# 이동 시간 및 대칭 비용 행렬 생성
travel_time_matrix = zeros(num_nodes, num_nodes)
cost_matrix = zeros(num_nodes, num_nodes)
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            travel_time_matrix[i,j] = calculate_travel_time(node_positions[i], node_positions[j]) * (1/2)
            cost_matrix[i,j] = travel_time_matrix[i,j] * 2 # 비용은 이동 시간에 비례한다고 가정
        end
    end
end

# Define the maximum route duration (here, we're setting a random example)
# max_route_duration = rand(50:100)

In [133]:
loads

12-element Vector{Int64}:
  1
  1
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
  0
  0

In [134]:
time_windows

12-element Vector{Tuple{Int64, Int64}}:
 (5, 52)
 (10, 30)
 (5, 54)
 (8, 57)
 (8, 43)
 (28, 75)
 (8, 36)
 (10, 37)
 (8, 80)
 (20, 32)
 (0, 0)
 (0, 100)

In [135]:
service_durations

12-element Vector{Int64}:
 4
 4
 2
 4
 2
 3
 3
 4
 3
 4
 0
 0

In [136]:
node_positions

12-element Vector{Vector{Int64}}:
 [0, 0]
 [9, 9]
 [3, 4]
 [2, 1]
 [9, 0]
 [8, 8]
 [6, 10]
 [7, 3]
 [2, 10]
 [8, 5]
 [6, 8]
 [6, 8]

In [137]:
cost_matrix

12×12 Matrix{Float64}:
  0.0  13.0  5.0   2.0   9.0  11.0  12.0  8.0  10.0  9.0  10.0  10.0
 13.0   0.0  8.0  11.0   9.0   1.0   3.0  6.0   7.0  4.0   3.0   3.0
  5.0   8.0  0.0   3.0   7.0   6.0   7.0  4.0   6.0  5.0   5.0   5.0
  2.0  11.0  3.0   0.0   7.0   9.0  10.0  5.0   9.0  7.0   8.0   8.0
  9.0   9.0  7.0   7.0   0.0   8.0  10.0  4.0  12.0  5.0   9.0   9.0
 11.0   1.0  6.0   9.0   8.0   0.0   3.0  5.0   6.0  3.0   2.0   2.0
 12.0   3.0  7.0  10.0  10.0   3.0   0.0  7.0   4.0  5.0   2.0   2.0
  8.0   6.0  4.0   5.0   4.0   5.0   7.0  0.0   9.0  2.0   5.0   5.0
 10.0   7.0  6.0   9.0  12.0   6.0   4.0  9.0   0.0  8.0   4.0   4.0
  9.0   4.0  5.0   7.0   5.0   3.0   5.0  2.0   8.0  0.0   4.0   4.0
 10.0   3.0  5.0   8.0   9.0   2.0   2.0  5.0   4.0  4.0   0.0   0.0
 10.0   3.0  5.0   8.0   9.0   2.0   2.0  5.0   4.0  4.0   0.0   0.0

In [159]:
travel_time_matrix[4,8]

2.5

# Mathematical Modeling

Paolo Toth, Daniele Vigo, "The Vehicle Routing Problem - Ch9. VRP with Pickup and Delivery", pp.225-242

<img src="PDTSPTW_formulation.jpg" alt="Mathematical modeling" width="500">

In [141]:
# Initialize the model
model = Model(Gurobi.Optimizer)

# Define decision variables
@variable(model, x[1:num_nodes, 1:num_nodes]>=0, Bin)
@variable(model, service_start_time[1:num_nodes] >=0)
@variable(model, load_after_node[1:num_nodes] >=0)

# Define the objective function to minimize the total cost
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes)) # 9.1

# Define the constraints
# Each pickup and delivery match
for i in P
    @constraint(model, sum(x[i, j] for j in vcat(P, D, depot_end)) == 1)  # 9.2
    @constraint(model, sum(x[i, j] for j in vcat(P, D)) - sum(x[j,i+num_requests] for j in vcat(P, D)) == 0) # 9.3
end

# Leave from and return to the depot
@constraint(model, sum(x[depot_start, j] for j in vcat(P, depot_end)) == 1)  # 9.4
@constraint(model, sum(x[i, depot_end] for i in vcat(D, depot_start)) == 1)  # 9.6

for j in vcat(P, D)
    @constraint(model, sum(x[i, j] for i in vcat(P, D, depot_start)) - sum(x[j, i] for i in vcat(P, D, depot_end)) == 0) # 9.5
end

# Time window constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        @constraint(model, (service_start_time[i] + service_durations[i] + travel_time_matrix[i,j] - service_start_time[j]) * x[i,j] <= 0) # 9.7
    end
end

for i in 1:num_nodes
    @constraint(model, service_start_time[i] >= time_windows[i][1]) # 9.8   
    @constraint(model, service_start_time[i] <= time_windows[i][2]) # 9.8
end

for i in P
    @constraint(model, service_start_time[i] + travel_time_matrix[i,i+num_requests] <= service_start_time[i+num_requests]) # 9.9
end

# Vehicle load and capacity constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        @constraint(model, (load_after_node[i] + loads[j] - load_after_node[j]) * x[i,j] == 0) # 9.10
    end
end

for i in P
    @constraint(model, load_after_node[i] >= loads[i]) # 9.11
    @constraint(model, load_after_node[i] <= vehicle_capacity) # 9.11
end

for i in D
    @constraint(model, load_after_node[i] >= 0) # 9.12
    @constraint(model, load_after_node[i] <= vehicle_capacity - loads[i-num_requests]) # 9.12
end

@constraint(model, load_after_node[depot_start] == 0) # 9.13

# Solve the problem
optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 72 rows, 168 columns and 412 nonzeros
Model fingerprint: 0xfe45fcac
Model has 288 quadratic constraints
Variable types: 24 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve added 221 rows and 0 columns
Presolve removed 0 rows and 37 columns
Presolve time: 0.00s
Presolved: 415 rows, 212 columns, 2049 nonzeros
Presolved model has 42 SOS constraint(s)
Variable types: 82 continuous, 130 integer (129 binary)

Root relaxation: objective 3.516667

# Solve the model

In [170]:
# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    println("An optimal solution has been found:")
    println("Optimal objective value: ", objective_value(model))
    
    # Route reconstruction
    route = [depot_start]
    current_node = depot_start

    while current_node != depot_end
        for i in 1:num_nodes
            for j in 1:num_nodes
                if value(x[i,j]) > 0.9
                    if i == current_node
                        println("Node $i to node $j")
                        push!(route, j)
                        current_node = j
                    end
                end
            end
        end
    end
    
    println("Vehicle Route: ", route)
    
    # Print the service start times
    println("\nService start times:")
    for i in 1:num_nodes
        println("Node $i: ", value(service_start_time[i]))
    end
    
    # Print the vehicle loads
    println("\nVehicle loads after each node:")
    for i in 1:num_nodes
        println("Node $i: ", value(load_after_node[i]))
    end
    
else
    println("No optimal solution found. Status: ", termination_status(model))
end

An optimal solution has been found:
Optimal objective value: 52.0
Node 11 to node 2
Node 2 to node 7
Node 7 to node 3
Node 3 to node 5
Node 5 to node 10
Node 10 to node 8
Node 8 to node 4
Node 4 to node 1
Node 1 to node 9
Node 9 to node 6
Node 6 to node 12
Vehicle Route: [11, 2, 7, 3, 5, 10, 8, 4, 1, 9, 6, 12]

Service start times:
Node 1: 52.0
Node 2: 10.0
Node 3: 22.0
Node 4: 47.0
Node 5: 27.5
Node 6: 75.0
Node 7: 15.5
Node 8: 37.0
Node 9: 61.0
Node 10: 32.0
Node 11: 0.0
Node 12: 79.0

Vehicle loads after each node:
Node 1: 2.0
Node 2: 1.0
Node 3: 1.0
Node 4: 1.0
Node 5: 2.0
Node 6: 0.0
Node 7: 0.0
Node 8: 0.0
Node 9: 1.0
Node 10: 1.0
Node 11: 0.0
Node 12: 0.0


In [172]:
# # Save the model and solution
# JuMP.write_to_file(model, "example_model.lp")

# open("example_result.txt", "w") do file
#     if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
#         println(file, "An optimal solution has been found:")
#         println(file, "Optimal objective value: ", objective_value(model))
        
#         # Route reconstruction
#         route = [depot_start]
#         current_node = depot_start
    
#         while current_node != depot_end
#             for i in 1:num_nodes
#                 for j in 1:num_nodes
#                     if value(x[i,j]) > 0.9
#                         if i == current_node
#                             println(file, "Node $i to node $j")
#                             push!(route, j)
#                             current_node = j
#                         end
#                     end
#                 end
#             end
#         end
        
#         println(file, "Vehicle Route: ", route)
        
#         # Print the service start times
#         println(file, "\nService start times:")
#         for i in 1:num_nodes
#             println(file, "Node $i: ", value(service_start_time[i]))
#         end
        
#         # Print the vehicle loads
#         println(file, "\nVehicle loads after each node:")
#         for i in 1:num_nodes
#             println(file, "Node $i: ", value(load_after_node[i]))
#         end
        
#     else
#         println(file, "No optimal solution found. Status: ", termination_status(model))
#     end
# end